In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


# Import libraries

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [0]:
path = "/content/gdrive/My Drive/Folio"
folder = os.listdir(path)
folder[0].split(' ')[0]

FileNotFoundError: ignored

# Sort folder

In [0]:
img = cv2.imread(os.path.join(path,folder[0]))
img.shape

(4128, 2322, 3)

A class for storing and loading images

In [2]:
#600x32x32x3
#ImageHolder.labels
class ImageHolder:
  def __init__(self):
    self.img = np.empty((0,128,128,3)).astype('uint8')
    self.labels = np.empty((0))
  def load_data(self,path):
    count = 0
    folder = os.listdir(path)
    folder.sort(key=lambda x:int(x[:2]))
    for filename in folder:
      print(count)
      count+=1
      temp_img = np.array([cv2.resize(cv2.imread(os.path.join(path,filename)),(128,128))]).astype('uint8')      
      self.img = np.r_[self.img,temp_img] 
      self.labels = np.r_[self.labels,int(filename.split(' ')[0])]
    return self

Tools for save and load object

In [4]:
def save_obj(var,file_name):
    import pickle
    with open(file_name+'.pkl','wb') as handle:
        pickle.dump(var,handle,protocol=pickle.HIGHEST_PROTOCOL)
  
def load_obj(file_name):
    import pickle
    with open(file_name+'.pkl','rb') as handle:
        return pickle.load(handle)


In [4]:
ImgHolder = ImageHolder().load_data(path)
save_obj(ImgHolder,'ImgHolder')

NameError: name 'path' is not defined

In [0]:
save_obj(ImgHolder,'ImgHolder')

In [5]:
ImgHolder = load_obj('ImgHolder')

# Extract features with ResNet50

In [6]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
import numpy as np

pretrained_module = ResNet50(weights='imagenet', include_top=False)
extracted_features = pretrained_module.predict(ImgHolder.img)
num_data = extracted_features.shape[0] 
feature_size = np.prod(extracted_features.shape[1:])
extracted_features = extracted_features.reshape(num_data,feature_size)

from sklearn.preprocessing import Normalizer

extracted_features = Normalizer().fit_transform(extracted_features)


Instructions for updating:
Colocations handled automatically by placer.


/home/truongphatuit/.conda/envs/tf2/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


# Softmax

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(extracted_features, ImgHolder.labels, test_size=100, random_state=42)

model = LogisticRegression(multi_class='multinomial',solver="lbfgs")
model.fit(X_train,y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [12]:
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)


0.98

# K-nearest neighbors

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(extracted_features, ImgHolder.labels, test_size=100, random_state=42)

model = KNeighborsClassifier()
model.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [21]:
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)


0.98

# MultinomialNB

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(extracted_features, ImgHolder.labels, test_size=100, random_state=42)

model = MultinomialNB(alpha=0.01)
model.fit(X_train,y_train)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [60]:
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)


0.98

## GuassianNB

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(extracted_features, ImgHolder.labels, test_size=100, random_state=42)

model = GaussianNB()
model.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [62]:
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

0.96

# Decision Tree

In [132]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(extracted_features, ImgHolder.labels, test_size=100, random_state=42)

model = DecisionTreeClassifier()
model.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [133]:
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

0.62

# Random Forest

In [118]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(extracted_features, ImgHolder.labels, test_size=100, random_state=42)

model = RandomForestClassifier(min_samples_leaf=7,random_state=8)
model.fit(X_train,y_train)

/home/truongphatuit/.conda/envs/tf2/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=8, verbose=0, warm_start=False)

In [119]:
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

0.92